In [3]:
import pandas as pd
import requests

from datetime import datetime, timedelta
from io import StringIO

In [4]:
VOLTILITY_DOWNLOAD_URL = "https://nseindia.com/archives/nsccl/volt/CMVOLT_{}.CSV"
NIFTY_100_URL = "https://www.nseindia.com/content/indices/ind_nifty100list.csv"

In [5]:
def get_download_date(today=False):
    date_today = datetime.today().date()
    if today:
        return date_today.strftime("%d%m%Y")
    else:
        days_before_to_pull = 0
        if date_today.weekday() > 4:
            days_before_to_pull = date_today.weekday() - 4
        elif date_today.weekday() == 0:
            days_before_to_pull = 3
        return (date_today - timedelta(days=days_before_to_pull)).strftime("%d%m%Y")

In [6]:
def get_nifty_100():
    res = requests.get(NIFTY_100_URL)
    return pd.read_csv(StringIO(res.text))

In [7]:
def get_volatility_data(today=False):
    download_date = get_download_date(today=today)
    print("Volatility Download Date: {}".format(download_date))
    res = requests.get(VOLTILITY_DOWNLOAD_URL.format(download_date))
    return pd.read_csv(StringIO(res.text))

In [8]:
nifty_100 = get_nifty_100()
volatility = get_volatility_data()

Volatility Download Date: 07062019


In [9]:
volatility = volatility.iloc[:,[1,-2,-1]]
volatility.columns = ["Symbol", "DailyVolatility", "YearlyVolatility"]

In [10]:
volatility.loc[:, "YearlyVolatility"] = volatility.loc[:,"YearlyVolatility"].apply(lambda x: x * 100)
volatility.loc[:, "DailyVolatility"] = volatility.loc[:,"DailyVolatility"].apply(lambda x: x * 100)

In [11]:
volatility = volatility.merge(nifty_100[["Symbol"]], on="Symbol", how="inner").sort_values(
    by="DailyVolatility", ascending=False)

In [13]:
volatility.head(30)

,Symbol,DailyVolatility,YearlyVolatility
99,YESBANK,5.26,100.49
100,ZEEL,4.03,76.99
50,IDEA,3.67,70.12
46,IBULHSGFIN,3.60,68.78
31,GAIL,3.37,64.38
13,BANKBARODA,3.30,63.05
52,INDUSINDBK,3.27,62.47
74,PAGEIND,3.21,61.33
86,SRTRANSFIN,3.09,59.03
88,TATAMOTORS,3.05,58.27
